In [1]:
import os
import sys
import random
import threading
import numpy as np
import pythreejs as three
import ipywidgets as widgets
from IPython.display import display
sys.path.insert(0, os.path.abspath(".."))
from Simulators_and_Solvers.cube_simulator_full import CubeTracker, CubeColorizer

#-----------------------------------------------------------------------
# Helper Function for Coordinate Mapping
#-----------------------------------------------------------------------
def map_visual_to_logical(vx, vy, vz):
    """Maps visual coordinates (-1, 0, 1) to logical array indices (0, 1, 2)."""
    logical_i = vx + 1
    logical_j = vy + 1
    logical_k = vz + 1
    return (logical_i, logical_j, logical_k) # Example output

#-----------------------------------------------------------------------
# The Main 2D Visualizer Class (Using pythreejs)
#-----------------------------------------------------------------------
class CubeVisualizer2D_pythreejs:
    def __init__(self, figsize=(500, 400), facelet_size=30.0, gap=2.0):
        if CubeTracker is None or CubeColorizer is None:
             raise RuntimeError("Simulator classes not loaded.")

        self.colorizer = CubeColorizer()
        self.tracker = self.colorizer.cube_tracker
        self.fig_width, self.fig_height = figsize
        self.facelet_size = facelet_size
        self.gap = gap
        self.cell_size = facelet_size + gap

        # --- Layout Definition ---
        self.grid_positions = {
            'Z': (3, 6),  # White face (Up)
            'x': (0, 3),  # Orange face (Left)
            'y': (3, 3),  # Green face (Front)
            'X': (6, 3),  # Red face (Right)
            'Y': (9, 3),  # Blue face (Back)
            'z': (3, 0),  # Yellow face (Down)
        }
        self.min_grid_x = min(pos[0] for pos in self.grid_positions.values())
        self.max_grid_x = max(pos[0] + 3 for pos in self.grid_positions.values())
        self.min_grid_y = min(pos[1] for pos in self.grid_positions.values())
        self.max_grid_y = max(pos[1] + 3 for pos in self.grid_positions.values())
        self.layout_width_grid = self.max_grid_x - self.min_grid_x
        self.layout_height_grid = self.max_grid_y - self.min_grid_y
        self.layout_width_px = self.layout_width_grid * self.cell_size
        self.layout_height_px = self.layout_height_grid * self.cell_size
        self.center_x_px = (self.min_grid_x * self.cell_size + self.layout_width_px / 2.0)
        self.center_y_px = (self.min_grid_y * self.cell_size + self.layout_height_px / 2.0)

        # --- Core 3JS Components ---
        cam_half_width = self.layout_width_px * 0.6
        cam_half_height = self.layout_height_px * 0.6
        self.camera = three.OrthographicCamera(
            left=-cam_half_width, right=cam_half_width,
            top=cam_half_height, bottom=-cam_half_height,
            near=0.1, far=1000
        )
        self.camera.position = [self.center_x_px, self.center_y_px, 10]
        self.camera.lookAt([self.center_x_px, self.center_y_px, 0])
        self.scene = three.Scene(background='lightgray')
        self.scene.add(three.AmbientLight(color='#FFFFFF', intensity=1.0))

        # --- Materials ---
        self._create_materials()

        # --- Build 2D Model ---
        self.facelet_meshes = {}
        self.cube_group = self._build_cube_model()
        self.scene.add(self.cube_group)

        # --- Renderer ---
        self.renderer = three.Renderer(camera=self.camera, scene=self.scene,
                                       controls=[], width=self.fig_width, height=self.fig_height,
                                       antialias=True)

        # --- GUI Elements ---
        self._create_gui()
        self._setup_callbacks()

        # --- Initial State Update ---
        self.tracker.apply_moves('N')
        self.update_visual_state()
        self.update_history()
        self.solution_text.value = "" # Initialize solution text

        # --- Display ---
        display(self.ui)

    def _create_materials(self):
        """Creates pythreejs materials."""
        self.materials = {
            "White":  three.MeshBasicMaterial(color='#FFFFFF', side='DoubleSide'),
            "Yellow": three.MeshBasicMaterial(color='#FBFB45', side='DoubleSide'),
            "Blue":   three.MeshBasicMaterial(color='#4095FE', side='DoubleSide'),
            "Green":  three.MeshBasicMaterial(color='#00EF2A', side='DoubleSide'),
            "Red":    three.MeshBasicMaterial(color='#FF0000', side='DoubleSide'),
            "Orange": three.MeshBasicMaterial(color='#FF8800', side='DoubleSide'),
            "Black":  three.MeshBasicMaterial(color='#333333', side='DoubleSide'),
        }
        self.edges_material = three.LineBasicMaterial(color='#000000', linewidth=1)

    def _build_cube_model(self):
        """Builds the 2D unfolded layout using PlaneGeometry."""
        main_group = three.Group()
        facelet_geom = three.PlaneGeometry(self.facelet_size, self.facelet_size)
        edges_geom = three.EdgesGeometry(facelet_geom)

        for direction, (base_grid_x, base_grid_y) in self.grid_positions.items():
             for r in range(3):
                 for c in range(3):
                    plot_grid_x = base_grid_x + c
                    plot_grid_y = base_grid_y + (2 - r)
                    center_x = (plot_grid_x + 0.5) * self.cell_size
                    center_y = (plot_grid_y + 0.5) * self.cell_size

                    mat = self.materials["Black"] # Default
                    facelet_mesh = three.Mesh(facelet_geom, mat)
                    facelet_mesh.position = [center_x, center_y, 0]

                    edges = three.LineSegments(edges_geom, self.edges_material)
                    edges.position = facelet_mesh.position

                    tile_group = three.Group()
                    tile_group.add(facelet_mesh)
                    tile_group.add(edges)

                    main_group.add(tile_group)
                    self.facelet_meshes[(direction, r, c)] = facelet_mesh

        return main_group

    def update_visual_state(self):
        """Updates the materials of the facelet meshes."""
        new_colors = self.colorizer.update_colors()
        face_id_maps = self.tracker.cube_current_faces_with_ids

        for (direction, r, c), facelet_mesh in self.facelet_meshes.items():
            try:
                id_array = face_id_maps.get(direction)
                if id_array is None: continue
                piece_id = id_array[r, c]
                piece_color_list = new_colors.get(piece_id)
                if piece_color_list is None: continue

                color_idx = self.colorizer.direction__color_idx_map.get(direction)
                if color_idx is not None and color_idx < len(piece_color_list):
                    color_name = piece_color_list[color_idx]
                    new_material = self.materials.get(color_name, self.materials["Black"])
                    facelet_mesh.material = new_material
                else:
                        facelet_mesh.material = self.materials["Black"]
            except Exception as e:
                    print(f"Error updating facelet ({direction},{r},{c}): {type(e).__name__} - {e}")

    def _create_gui(self):
        """Set up the UI controls with move buttons on the right."""
        # --- Create Buttons ---
        moves = ['F', 'f', 'B', 'b', 'U', 'u', 'D', 'd', 'L', 'l', 'R', 'r']
        self.move_buttons = {}
        for move in moves:
            button = widgets.Button(
                description=move, button_style='', tooltip=f'Apply {move} move',
                layout=widgets.Layout(width='40px', height='40px')
            )
            self.move_buttons[move] = button

        self.scramble_button = widgets.Button(
            description='Scramble', button_style='info', tooltip='Randomly scramble the cube',
            layout=widgets.Layout(width='auto'))
        self.reset_button = widgets.Button(
            description='Reset', button_style='warning', tooltip='Reset to solved state',
            layout=widgets.Layout(width='auto'))
        self.solve_button = widgets.Button(
            description='Solve', button_style='success', tooltip='Solve the cube (Not Implemented)',
            layout=widgets.Layout(width='auto'))
        self.apply_button = widgets.Button(
            description='Apply', tooltip='Apply custom move sequence',
            layout=widgets.Layout(width='auto'))

        # --- Create Text Fields (Adjusted Heights) ---
        textbox_height = '40px' # Reduced height
        self.history_text = widgets.Textarea(
            value='', placeholder='Move history appears here', disabled=True,
            layout=widgets.Layout(width='95%', height=textbox_height, overflow_y='auto') # Added overflow
        )
        # --- ADDED SOLUTION TEXTAREA ---
        self.solution_text = widgets.Textarea(
            value='', placeholder='Solver output appears here', disabled=True,
            layout=widgets.Layout(width='95%', height=textbox_height, overflow_y='auto', margin='5px 0 0 0') # Added top margin
        )
        # --- END ADDITION ---

        self.move_input = widgets.Text(
            placeholder='Enter sequence (e.g., FRBL)',
            layout=widgets.Layout(flex='1 1 auto', width='auto')
        )

        # --- Arrange Layout (Include solution_text) ---
        action_button_box = widgets.HBox(
            [self.scramble_button, self.reset_button, self.solve_button],
            layout=widgets.Layout(justify_content='space-around')
        )
        # --- MODIFIED left_panel VBox ---
        left_panel = widgets.VBox(
            [action_button_box, self.history_text, self.solution_text], # Added solution_text here
            layout=widgets.Layout(min_width='250px', margin='0 15px 0 0')
        )
        # --- END MODIFICATION ---

        move_button_rows = [
            widgets.HBox([self.move_buttons[m] for m in ['F', 'f', 'B', 'b']]),
            widgets.HBox([self.move_buttons[m] for m in ['U', 'u', 'D', 'd']]),
            widgets.HBox([self.move_buttons[m] for m in ['L', 'l', 'R', 'r']])
        ]
        move_button_box = widgets.VBox(move_button_rows)

        custom_move_box = widgets.HBox(
            [self.move_input, self.apply_button],
            layout=widgets.Layout(width='100%')
            )
        right_panel = widgets.VBox(
            [move_button_box, custom_move_box],
            layout=widgets.Layout(min_width='180px')
            )

        self.control_panel = widgets.HBox(
            [left_panel, right_panel],
            layout=widgets.Layout(margin='10px 0 0 0', justify_content='space-between')
            )

        self.ui = widgets.HBox([self.renderer, self.control_panel])


    def _setup_callbacks(self):
        """Set up the on_click callbacks for buttons."""
        def make_move_callback(move_code):
            def on_click(button):
                self.apply_move(move_code)
            return on_click

        for move_code, button in self.move_buttons.items():
            button.on_click(make_move_callback(move_code))

        self.scramble_button.on_click(self.scramble_cube)
        self.reset_button.on_click(self.reset_cube)
        self.solve_button.on_click(self.solve_cube)
        self.apply_button.on_click(lambda b: self.apply_custom_moves())

        self.move_input.observe(self._on_move_input_change, names='value')

    def apply_move(self, move):
        """Apply a single move to the cube and update."""
        try:
            self.tracker.apply_moves(move)
            self.update_visual_state()
            self.update_history()
            self.solution_text.value = "" # Clear solution text
        except ValueError as e:
            print(f"Error applying move {move}: {e}")
        except Exception as e:
             print(f"An unexpected error occurred applying move {move}: {e}")

    def _on_move_input_change(self, change):
        """Handle changes to the move input text box (currently inactive)."""
        # Primarily using Apply button now
        pass

    def apply_custom_moves(self):
        """Apply a custom sequence of moves from the text input."""
        moves = self.move_input.value
        valid_moves = True
        error_msg = ""
        if not moves: return

        for idx, move in enumerate(moves):
            if move not in self.tracker.move_map:
                valid_moves = False
                error_msg = f"Invalid move '{move}' at index {idx}"
                break

        if valid_moves:
            try:
                self.tracker.apply_moves(moves)
                self.update_visual_state()
                self.update_history()
                self.solution_text.value = "" # Clear solution text
                self.move_input.value = ""
                self.move_input.placeholder = 'Enter sequence (e.g., FRBL)'
            except Exception as e:
                error_msg = f"Error applying sequence: {e}"
                valid_moves = False

        if not valid_moves:
            original_placeholder = 'Enter sequence (e.g., FRBL)' # Store default
            self.move_input.placeholder = error_msg if error_msg else "Invalid sequence!"
            def reset_placeholder():
                if self.move_input.placeholder != original_placeholder:
                    self.move_input.placeholder = original_placeholder
            threading.Timer(2.5, reset_placeholder).start()

    def scramble_cube(self, button):
        """Randomly scramble the cube."""
        moves = list(self.move_buttons.keys())
        scramble_len = 20
        scramble = ''.join(random.choices(moves, k=scramble_len))
        self.tracker.apply_moves(scramble)
        self.update_visual_state()
        self.update_history()
        self.solution_text.value = "" # Clear solution text

    def reset_cube(self, button):
        """Reset the cube to solved state."""
        try:
            self.colorizer.cube_tracker = CubeTracker()
            self.tracker = self.colorizer.cube_tracker
            self.tracker.move_history = []
            self.update_visual_state()
            self.update_history()
            self.solution_text.value = "" # Clear solution text
        except Exception as e:
             print(f"Error resetting cube: {e}")

    def solve_cube(self, button):
        """Placeholder for solving the cube."""
        # Clear previous message immediately
        self.solution_text.value = "Solver not yet implemented!"
        # Define function to clear the message later
        def clear_solution_placeholder():
             if self.solution_text.value == "Solver not yet implemented!":
                 self.solution_text.value = ""
        # Set timer to clear it
        threading.Timer(2.5, clear_solution_placeholder).start()

    def update_history(self):
        """Update the move history display."""
        history = ''.join(self.tracker.move_history)
        self.history_text.value = history

visualizer_2d = CubeVisualizer2D_pythreejs(figsize=(600,450)) # Adjust size as needed

Successfully loaded tables: edge_distances, corner_distances, movements


KeyError: 0